## Sequence Learning Problem

The echo sequence prediction problem is a contrived problem for demonstrating the memory capability of the Vanilla LSTM. The task is that, given a sequence of random integers as input, to output the value of a random integer at a specific time input step that is not specified to the model.

For example, given the input sequence of random integers [5,3,2] and the chosen time step was the second value, then the expected output is 3. Technically, this is a sequence classification problem; it is formulated as a many-to-one prediction problem, where there are multiple input time steps and one output time step at the end of the sequence.

In [1]:
# importing required libraries

import numpy as np
from random import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [2]:
# generate a sequence of random numbers in [0, n_features]

def generate_sequence(length, n_features):
    return [np.random.randint(0, n_features - 1) for _ in range(length)]

In [3]:
# one hot encode sequence

def one_hot_encode(sequence, n_features):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_features)]
        # print('vector : ', vector)
        vector[value] = 1
        # print('vector[value] :', vector)
        encoding.append(vector)
    return np.array(encoding)

In [4]:
# decode a one hot encoded string

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [5]:
# generate random sequence

sequence = generate_sequence(3, 100)
print(sequence)

[73, 14, 25]


In [6]:
# one hot encode

encoded = one_hot_encode(sequence, 100)
print(encoded)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [7]:
# one hot decode

decoded = one_hot_decode(encoded)
print(decoded)

[73, 14, 25]


In [8]:
# Reshaping Sequence

# LSTM expect 3-dimensional matrix of samples, time steps, features

# generate one example for an LSTM

# out_index is the index of output

def generate_example(length, n_features, out_index):
    # generate sequence
    sequence = generate_sequence(length, n_features)
    
    # one hot encode
    encoded = one_hot_encode(sequence, n_features)
    
    # reshape sequence to be a 3D matrix
    X = encoded.reshape(1, length, n_features)
    
    # select the output
    y = encoded[out_index].reshape(1, n_features)
    
    return X, y

In [9]:
X, y = generate_example(25, 100, 2)

In [10]:
print('X shape :', X.shape)

X shape : (1, 25, 100)


In [11]:
print('y shape :', y.shape)

y shape : (1, 100)


In [12]:
# define the LSTM model

length = 5
n_features = 10
out_index = 2

model = Sequential()
model.add(LSTM(units=25, input_shape=(length, n_features)))
model.add(Dense(units=n_features, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 25)                3600      
_________________________________________________________________
dense (Dense)                (None, 10)                260       
Total params: 3,860
Trainable params: 3,860
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# Fit the model

import time
start_time = time.time()

# fit the model
for i in range(10000):
    X, y = generate_example(length, n_features, out_index)
    print('X :', X)
    print('y :', y)
    print('model.fit')
    model.fit(X, y, epochs=1, verbose=1)

print('Time Taken :', time.time() - start_time)

X : [[[0 0 0 0 1 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 1 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 1 0 0 0 0 0]]
model.fit
1/1 [==============================] - 8s 8s/step - loss: 2.3838 - accuracy: 0.0000e+00
X : [[[0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 1 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 2.2751 - accuracy: 0.0000e+00
X : [[[0 0 0 0 0 1 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]]]
y : [[0 0 1 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 12ms/step - loss: 2.2506 - accuracy: 0.0000e+00
X : [[[0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 1 0 0 0 0 0]]]
y : [[0 1 0 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 2.2197 - accurac

1/1 [==============================] - 0s 10ms/step - loss: 2.3952 - accuracy: 0.0000e+00
X : [[[0 0 0 0 0 0 0 0 1 0]
  [0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 1 0 0 0 0 0 0]]]
y : [[0 0 0 0 0 1 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 2.3458 - accuracy: 0.0000e+00
X : [[[0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]]]
y : [[0 0 0 0 0 0 0 0 1 0]]
model.fit
1/1 [==============================] - 0s 9ms/step - loss: 2.4074 - accuracy: 0.0000e+00
X : [[[0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 0 1 0 0 0 0 0 0]]]
y : [[0 0 0 0 0 1 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 2.3812 - accuracy: 0.0000e+00
X : [[[0 0 0 0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]]]
y : [[0 0 0 0 1 0 0 0 0

1/1 [==============================] - 0s 11ms/step - loss: 2.2009 - accuracy: 0.0000e+00
X : [[[0 1 0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]]]
y : [[0 0 0 1 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 2.3413 - accuracy: 0.0000e+00
X : [[[1 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 1 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 2.2926 - accuracy: 0.0000e+00
X : [[[0 0 0 0 0 0 0 1 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 0 1 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 2.2836 - accuracy: 0.0000e+00
X : [[[0 1 0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]]]
y : [[0 0 1 0 0 0 0 0 

X : [[[0 0 0 0 0 1 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]]]
y : [[1 0 0 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 2.3763 - accuracy: 0.0000e+00
X : [[[0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]]]
y : [[0 1 0 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 2.1943 - accuracy: 1.0000
X : [[[0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 1 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 0 0 0 1 0 0]]
model.fit
1/1 [==============================] - 0s 12ms/step - loss: 2.3104 - accuracy: 0.0000e+00
X : [[[0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]]]
y : [[0 0 1 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 2.1150 - accuracy:

1/1 [==============================] - 0s 13ms/step - loss: 2.3522 - accuracy: 0.0000e+00
X : [[[0 0 0 0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 0 0 1 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 2.3010 - accuracy: 0.0000e+00
X : [[[0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]]]
y : [[1 0 0 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 2.3365 - accuracy: 0.0000e+00
X : [[[0 0 0 1 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 0 0 1 0 0]]]
y : [[0 0 1 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 2.2123 - accuracy: 1.0000
X : [[[0 0 0 1 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 1 0 0 0 0 0 0 0]]]
y : [[0 0 0 1 0 0 0 0 0 0]

...

1/1 [==============================] - 0s 10ms/step - loss: 0.0486 - accuracy: 1.0000
X : [[[0 0 0 0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 0 1 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 0 0 0 0 1 0]]
model.fit
1/1 [==============================] - 0s 12ms/step - loss: 0.0185 - accuracy: 1.0000
X : [[[1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]]]
y : [[1 0 0 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0510 - accuracy: 1.0000
X : [[[0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]]]
y : [[0 0 0 1 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 7.8004e-04 - accuracy: 1.0000
X : [[[0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 1 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]]]
y : [[0 0 1 0 0 0 0 0 0 0]]
model.

1/1 [==============================] - 0s 10ms/step - loss: 0.0050 - accuracy: 1.0000
X : [[[0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 0 0 0 1 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0063 - accuracy: 1.0000
X : [[[0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 1 0 0 0 0 0 0]]]
y : [[0 0 1 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 0.0031 - accuracy: 1.0000
X : [[[0 0 1 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]]]
y : [[0 0 0 0 1 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 0.0015 - accuracy: 1.0000
X : [[[0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]]]
y : [[0 1 0 0 0 0 0 0 0 0]]
model.fit


1/1 [==============================] - 0s 10ms/step - loss: 0.0029 - accuracy: 1.0000
X : [[[0 0 0 0 0 0 0 1 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 0 0 0 0 1 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0054 - accuracy: 1.0000
X : [[[1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]]]
y : [[0 0 0 0 0 0 0 0 1 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0122 - accuracy: 1.0000
X : [[[1 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 1 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0138 - accuracy: 1.0000
X : [[[0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]]]
y : [[0 0 0 0 0 0 0 1 0 0]]
model.fit


1/1 [==============================] - 0s 11ms/step - loss: 0.1306 - accuracy: 1.0000
X : [[[0 0 0 0 0 0 1 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]]]
y : [[0 0 0 0 0 0 0 1 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0046 - accuracy: 1.0000
X : [[[1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]]]
y : [[0 0 0 1 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0053 - accuracy: 1.0000
X : [[[0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]]]
y : [[0 0 0 0 1 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0092 - accuracy: 1.0000
X : [[[0 0 0 1 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]]]
y : [[0 0 0 0 0 0 0 1 0 0]]
model.fit


1/1 [==============================] - 0s 10ms/step - loss: 0.0030 - accuracy: 1.0000
X : [[[0 0 0 0 0 0 1 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]]]
y : [[0 0 0 1 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 11ms/step - loss: 0.0015 - accuracy: 1.0000
X : [[[1 0 0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]]]
y : [[1 0 0 0 0 0 0 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0030 - accuracy: 1.0000
X : [[[0 1 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 1 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0]]]
y : [[0 0 0 0 0 0 1 0 0 0]]
model.fit
1/1 [==============================] - 0s 10ms/step - loss: 0.0060 - accuracy: 1.0000
X : [[[0 0 0 0 0 0 0 0 1 0]
  [0 0 0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0 0 0 0]]]
y : [[0 0 0 0 0 0 0 1 0 0]]
model.fit


In [15]:
# Evaluate the model

correct = 0
for i in range(100):
    X, y = generate_example(length, n_features, out_index)
    yhat = model.predict(X)
    if one_hot_decode(yhat) == one_hot_decode(y):
        correct += 1

print("Accuracy : %f" %((correct/100) * 100.0))

Accuracy : 100.000000


In [16]:
# Make prediction with the model

# prediction on new data
X, y = generate_example(length, n_features, out_index)
yhat = model.predict(X)

print("Sequence : %s" % [one_hot_decode(x) for x in X])
print("Expected : %s" % one_hot_decode(y))
print("Predicted : %s" % one_hot_decode(yhat))

Sequence : [[8, 1, 5, 3, 3]]
Expected : [5]
Predicted : [5]


In [17]:
# Make prediction with the model

# prediction on new data
X, y = generate_example(length, n_features, out_index)
yhat = model.predict(X)

print("Sequence : %s" % [one_hot_decode(x) for x in X])
print("Expected : %s" % one_hot_decode(y))
print("Predicted : %s" % one_hot_decode(yhat))

Sequence : [[0, 1, 4, 6, 2]]
Expected : [4]
Predicted : [4]
